In [1]:
import requests
import time
import datetime
import re
from bs4 import BeautifulSoup
from IPython.display import display, HTML

In [2]:
#Letterboxd
def pelicula(titulo):  
    nombre_formateado=titulo.lower().replace(' ','-')
    url='https://letterboxd.com/film/'+nombre_formateado
    return url
def buscarInfo(soup):
    tituloPeli = soup.find('meta', property='og:title')['content']
    director_element = soup.find('span', class_='prettify')
    director_name = director_element.text.strip()
    descripcion = soup.find('meta', property='og:description')['content']
    promedio_puntuacion = soup.find('meta', {'name': 'twitter:data2'})['content']
    return tituloPeli,director_name,descripcion,promedio_puntuacion
def coger_nota(promedio_puntuacion):
    try:
        # Realizar split y convertir el primer elemento a float
        nota = float(promedio_puntuacion.split()[0])
        
        # Multiplicar por 2
        nota_multiplicada = nota * 2

        return nota_multiplicada
    except ValueError:
        # Manejar posibles errores de conversión
        print("Error al convertir la nota a número.")
        return None
def coger_critica(soup):
    # Supongamos que las críticas están dentro de un div con la clase "criticas"
    criticas_section = soup.find('section', class_='film-recent-reviews')

    if criticas_section:
        # Extraer todas las críticas (puedes limitar la cantidad según tus necesidades)
        criticas = criticas_section.find_all('p')

        # Devolver la primera crítica (puedes ajustar esto según tus necesidades)
        if criticas:
           
            return '\n'.join([critica.text.strip() for critica in criticas])


    return None

In [3]:
#IMDB
imdb='https://www.imdb.com'
headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',}
def buscar(titulo):
    tit_format=titulo.lower().replace(' ','%20')
    urlb=imdb+f'/find/?q={tit_format}'
    b=requests.get(urlb, headers=headers)
    sopa=BeautifulSoup(b.content, 'html.parser')
    entradas=sopa.find_all('a', class_='ipc-metadata-list-summary-item__t')
    for entrada in entradas:
        if entrada is not None:
            return imdb+entrada.get('href') #título y url
    return None

def coger_notaI(url):  #coge la nota de una pelicula, previamente ha sido seleccionada la url de la pelicula
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    nota=sopa.find('span', {'class': 'sc-bde20123-1 cMEQkK'})
    return float(nota.text)


def coger_criticaI(url):  #En ocasiones las críticas no tienen nota, hay que solucionar eso+
    r=url.split('/')
    del r[-1]
    cleaned_url='/'.join(r)
    reviews_url=cleaned_url+'/reviews'
    c=requests.get(reviews_url, headers=headers)
    sopa=BeautifulSoup(c.content, 'html.parser')
    padre=sopa.find('span', class_='rating-other-user-rating')
    if padre is not None:
        nota=padre.find('span', class_=None)
        calif=nota.text+'/10'
    else:
        calif='Unrated'
    titulo_critica=sopa.find('a', class_='title')
    critica=sopa.find('div', class_='text show-more__control')
    return calif, titulo_critica.text, critica.text
def video(url):
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    v=sopa.find_all('a', class_='ipc-lockup-overlay ipc-focusable')
    for vid in v:
        if '/video' in vid['href']:
            return imdb+vid['href']
    return 'Not Found'

In [4]:
#FilmAffinity
from IPython.display import display, Image
from io import BytesIO
def buscar_peliF(titulos):
    año=titulos.rsplit(maxsplit=1)[-1]
    titulo=titulos.rsplit(maxsplit=1)[0]
    tit_format=titulo.lower().replace(' ','+')
    urlb=f'https://www.filmaffinity.com/es/search.php?stext={tit_format}'
    b=requests.get(urlb, headers=headers)
    sopa=BeautifulSoup(b.content, 'html.parser')
    all_div_etiq=sopa.find_all('div', class_='se-it mt')
    for div in all_div_etiq:
        d=div.find('div', class_='ye-w')
        d2=div.find('div', class_='movie-card mc-flex movie-card-1')
        if d.text==año:
            a=div.find('a')
            return a['href'],d2['data-movie-id']
    
def coger_notaF(url):
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    nota = sopa.find(id="movie-rat-avg")
    valor = nota.get_text(strip=True)
    nota=valor.replace(',','.')
    return float(nota)
def coger_criticaF(url,id):
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    critica_1=sopa.find('div',itemprop='reviewBody')
    urlc=f'https://www.filmaffinity.com/es/reviews/1/{id}.html'
    src=requests.get(urlc, headers=headers)
    sopa2=BeautifulSoup(src.content, 'html.parser')
    critica_2=sopa2.find('div', class_='review-text1')
    nota=sopa2.find('div', class_='user-reviews-movie-rating')
    return  critica_1.text,critica_2.text, nota.text
def imagen(url):
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    img= sopa.find('img', itemprop="image")
    urli=img['src']
    response = requests.get(urli)
    display(Image(response.content))
def trailer(urlf):
    id = re.search(r'\d+', urlf)
    if id:
        id=id.group()
        urlT=f'https://www.filmaffinity.com/es/evideos.php?movie_id={id}'
        t=requests.get(urlT, headers=headers)
        sopa=BeautifulSoup(t.content, 'html.parser')
        trailer=sopa.find('iframe')['src']
        return trailer

In [5]:
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
from bs4 import BeautifulSoup

def buscarPeli(nombre_pelicula):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
    }

    # Letterboxd
    url_letterboxd = pelicula(nombre_pelicula)
    a = requests.get(url_letterboxd, headers=headers)
    soup = BeautifulSoup(a.content, 'html.parser')

    urlf, idf = buscar_peliF(nombre_pelicula)
    urlb=buscar(nombre_pelicula)
    # Extraer información de los metadatos

    tituloPeli, director_name, descripcion, promedio_puntuacion = buscarInfo(soup)

    # Presentar la información con formato
    display(HTML(f'<h1 style="color:#4285f4;">{tituloPeli}</h1>'))
    display(HTML(f'<h2>Director: {director_name}</h2>'))
    display(HTML(f'<p>{descripcion}</p>'))
    imagen(urlf)
    print(video(urlb))

    # Letterboxd
    display(HTML('<h2>LETTERBOXD</h2>'))
    display(HTML(f'<p>Nota de Letterboxd: <strong>{coger_nota(promedio_puntuacion)}</strong></p>'))
    display(HTML('<h3>Críticas de usuarios de Letterboxd:</h3>'))
    
    # Separar las críticas de usuarios en una lista
    criticas_letterboxd = coger_critica(soup)
    if criticas_letterboxd:
        criticas_list = criticas_letterboxd.split('\n')
        for critica in criticas_list:
            display(HTML(f'<p>{critica}</p>'))

    # IMDb
    display(HTML('<h2>IMDB</h2>'))
    url_imdb = buscar(nombre_pelicula)
    display(HTML(f'<p>Nota de IMDB: <strong>{coger_notaI(url_imdb)}</strong></p>'))
    d, e, f = coger_criticaI(url_imdb)
    display(HTML(f'<h3>Críticas de IMDB:</h3>'))
    display(HTML(f'<p>{d} - {e} - {f}</p>'))

    # FilmAffinity
    display(HTML('<h2>FILMAFFINITY</h2>'))
    url_filmaffinity, id_filmaffinity = buscar_peliF(nombre_pelicula)
    display(HTML(f'<p>Nota de FilmAffinity: <strong>{coger_notaF(url_filmaffinity)}</strong></p>'))
    g, h, i = coger_criticaF(url_filmaffinity, id_filmaffinity)
    display(HTML(f'<h3>Crítica de FilmAffinity:</h3>'))
    display(HTML(f'<p>Crítica de experto: <br>{g} <br><br> Crítica de usuario: <br>{h}</p>'))

    

    # NOTAMEDIA
    nota_media = (coger_nota(promedio_puntuacion) + coger_notaI(url_imdb) + coger_notaF(url_filmaffinity)) / 3
    display(HTML('<h1 style="color:#4285f4;">Nota Media</h1>'))
    display(HTML(f'<p style="font-size: 18px;">La nota media de las tres páginas web es: <strong>{nota_media:.2f}</strong></p>'))

    #TRAILER
    video_url = trailer(url_filmaffinity)

    display(HTML(f'<iframe width="560" height="315" src={video_url} frameborder="0" allowfullscreen></iframe>'))
# ...


# Widget de entrada
input_pelicula = widgets.Text(value='', description='Nombre de la película:')
boton_buscar = widgets.Button(description='Buscar')

# Función para manejar el evento de clic en el botón
def on_button_click(b):
    buscarPeli(input_pelicula.value)

# Asocia la función al evento de clic en el botón
boton_buscar.on_click(on_button_click)

# Muestra los widgets con formato
display(HTML('<h1 style="color:#4285f4;">Web Scraping App</h1>'))
display(HTML('<p>Introduce el nombre de la película y haz clic en "Buscar".</p>'))
display(input_pelicula)
display(boton_buscar)


Text(value='', description='Nombre de la película:')

Button(description='Buscar', style=ButtonStyle())